In [1]:
from fl.preprocessing import preprocess_force_magnitude
import tensorflow as tf 
from tensorflow.keras.models import load_model

model = load_model("../models/force_prediction_model.h5")

2024-05-23 07:39:19.268966: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 07:39:19.457164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-23 07:39:19.457222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-23 07:39:19.491250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-23 07:39:19.564622: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 07:39:19.565270: I tensorflow/core/platform/cpu_feature_guard.cc:1

## WARNING!
**Make sure you have TensorFlow version 2.15!**
```
pip install tensorflow==2.15
```
It is necessary to have Python version 3.11.

For this challenge, **we are stepping out of the federated learning framework**. We are also moving away from MNIST image classification to focus on a different model: a neural network capable of predicting the power (speed) of a sailboat based on acceleration in x, y, and z.

A data preparation step is performed to calculate the amplitude (the squared norm) and to remove unnecessary data. I’ve provided both versions for reference, though this shouldn't be important for solving the challenge.

This is a **regression** model, meaning it doesn’t end with a sigmoid, but with nothing! All values are possible as outputs. Even though in practice, I use it with a notion of class, either 25 or 50. If the model's output is $x$, the predicted class is:
- 25 if $|25 - x|$ < $|50 - x|$
- 50 otherwise

**The goal of the challenge is to modify two weights so that the model predicts all class 25 as 50 and vice versa.**

You have four data examples in `data/example_force_...`. Good luck!

In [2]:
examples = ["25a", "25b", "50a", "50b"]
values = {example: tf.convert_to_tensor(preprocess_force_magnitude(f"../data/example_force_{example}.csv").to_numpy()[:, 0].reshape(1, 50)) for example in examples}
predictions = {example: model.predict(values[example])[0][0] for example in examples}
predictions

1/1 [==============================] - 0s 16ms/step


{'25a': 24.904825, '25b': 25.188284, '50a': 55.799343, '50b': 46.487675}

In [3]:
weights = model.get_weights()

# Structure of our neural network, classic: Dense + ReLU
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5377 (21.00 KB)
Trainable params: 5377 (21.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Flag Retrieval

In [ ]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

In [ ]:
d = {
    "position_1": [-4, 10, 25],  # For example: first weight to modify at layer -4 and position (10, 25)
    "value_1": 0.066,  # New value 
    "position_2": [-1, 0],  # Layer -1 is a bias layer, so only one coordinate needs to be provided
    "value_2": 0.0017
}
rq.post(URL + "/challenges/4", json=d).json()["message"]